In [1]:
f = open('name.txt','r')
name = f.read().split("\n")

name_dict = {}
for i in name:
    key, value = i.split(":")
    name_dict[key] = value

name_dict

{'0': 'luokuofeng', '1': 'MM', '2': 'leeweiwang'}

In [2]:
#即時串流影像人臉偵測
import dlib
import cv2
import imutils

import numpy as np
from keras.models import Sequential 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D 
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64,
3), activation = 'relu'))

# Max Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Convolution
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))

# Max Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Flattening
classifier.add(Flatten())

# Fully Connected
classifier.add(Dense(units = 128, activation = 'relu')) 
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 3, activation = 'softmax'))

classifier.compile(optimizer = 'adam', 
                        loss ='categorical_crossentropy', 
                     metrics = ['accuracy'])
classifier.load_weights("SaveModel/face.h5")



# 開啟影片檔案
cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture(r'C:\Users\user\Desktop\thesis\Data_enhancement\1.film\风小筝 别丢下我不管 MV 原创.mp4')
# cap = cv2.VideoCapture(r'C:\Users\user\Desktop\thesis\Data_enhancement\1.film\超神奇臉部識別App之3D版的小伶！小伶玩具  Xiaoling toys.mp4')  


# Dlib 的人臉偵測器
detector = dlib.get_frontal_face_detector()



# 以迴圈從影片檔案讀取影格，並顯示出來
while(cap.isOpened()):
    ret, frame = cap.read()

  # 偵測人臉
    face_rects, scores, idx = detector.run(frame, 0)

  # 取出所有偵測的結果
    for i, d in enumerate(face_rects):
        x1 = d.left()
        y1 = d.top()
        x2 = d.right()
        y2 = d.bottom()
        #text = "%2.2f(%d)" % (scores[i], idx[i])

        cropped = frame[int(y1):int(y2),int(x1):int(x2)]
        image=cv2.resize(cropped,(64, 64),interpolation=cv2.INTER_CUBIC)
        image = np.expand_dims(image, axis = 0)
        
        label = str(classifier.predict_classes(image)[0])
        
        label = name_dict[label]
        
        
        # 以方框標示偵測的人臉
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4, cv2.LINE_AA)

        # 標示類別
        cv2.putText(frame, label, (x1, y1), cv2.FONT_HERSHEY_DUPLEX,
                0.7, (255, 255, 255), 1, cv2.LINE_AA)

      # 顯示結果
    cv2.imshow("Face Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Using TensorFlow backend.
